In [1]:
import plotly.graph_objects as go
import numpy as np
import torch

In [2]:
def plot_model(model, loss_params=None, data_params=None):
    # ランダムシードの設定
    torch.manual_seed(42)
    np.random.seed(42)

    def func(x):
        if type(x) == torch.Tensor:
            return torch.sin(x)
        return np.sin(x)
    
    if data_params is None:
        data_params = {}
    n_points = data_params.get('n_points', 50)
    noise_std = data_params.get('noise_std', 0.1)
    outlier_ratio = data_params.get('outlier_ratio', 0.03)
    

    # サイン波データの生成
    def generate_sine_data(n_points=50, noise_std=0.1, outlier_ratio=0.03, func=np.sin):
        x = np.linspace(0, 2 * np.pi, n_points)
        y = func(x)
        noise = np.random.normal(0, noise_std, size=y.shape)
        y_noisy = y + noise

        n_outliers = int(n_points * outlier_ratio)
        outlier_indices = np.random.choice(n_points, n_outliers, replace=False)
        y_noisy[outlier_indices] = 5 + np.random.uniform(-0.1, 0.1, n_outliers)

        return torch.tensor(x, dtype=torch.float64), torch.tensor(y_noisy, dtype=torch.float64)

    # データ生成
    train_x, train_y = generate_sine_data(n_points=n_points, noise_std=noise_std, outlier_ratio=outlier_ratio, func=func)

    # データを2D形式に変換
    train_x_2d = train_x.unsqueeze(-1).to(torch.float64)
    train_y_2d = train_y.unsqueeze(-1).to(torch.float64)

    # モデルを訓練
    model.fit(train_x_2d, train_y_2d, loss_params=loss_params)

    # テスト用データの生成
    test_x = torch.linspace(0, 2 * np.pi, 200).unsqueeze(-1).to(torch.float64)
    true_y = func(test_x)

    # 予測
    posterior = model.posterior(test_x)
    mean = posterior.mean
    variance = posterior.variance
    std = torch.sqrt(variance)

    # 信用区間
    lower_bound = mean - 2 * std
    upper_bound = mean + 2 * std

    # プロット作成
    fig = go.Figure()

    # 真の関数
    fig.add_trace(go.Scatter(
        x=test_x.squeeze().numpy(),
        y=true_y.squeeze().numpy(),
        mode='lines',
        name='True Function',
        line=dict(color='red')
    ))

    # 観測データ
    fig.add_trace(go.Scatter(
        x=train_x.numpy(),
        y=train_y.numpy(),
        mode='markers',
        name='Observations',
        marker=dict(color='blue')
    ))

    # 平均関数
    fig.add_trace(go.Scatter(
        x=test_x.squeeze().numpy(),
        y=mean.squeeze().detach().numpy(),
        mode='lines',
        name='Predictive Mean',
        line=dict(color='green')
    ))

    # 信用区間の塗りつぶし
    fig.add_trace(go.Scatter(
        x=np.concatenate([test_x.squeeze().numpy(), test_x.squeeze().numpy()[::-1]]),
        y=np.concatenate([upper_bound.squeeze().detach().numpy(), lower_bound.squeeze().detach().numpy()[::-1]]),
        fill='toself',
        fillcolor="green",
        line=dict(color='rgba(255, 165, 0, 0)'),
        opacity=0.2,
        name='Confidence Interval (2σ)',
        showlegend=True
    ))

    # プロットの装飾
    fig.update_layout(
        title="BNN Regression on Sine Function",
        xaxis_title="x",
        yaxis_title="y",
        legend=dict(orientation="h", y=-0.2),
        template="plotly_white"
    )

    fig.show()

In [3]:
from models.laplace import LaplaceBNN

n = 3

# モデルの初期化
args = {
    "regnet_dims": [64*n, 64*n, 64*n],
    "regnet_activation": "tanh",
    "prior_var": 1.0,
    "noise_var": 0.2,
    "iterative": True,
}
input_dim = 1
output_dim = 1
device = torch.device("cpu")

model = LaplaceBNN(args, input_dim=input_dim, output_dim=output_dim, device=device)


data_params = {
    "n_points": 50,
    "noise_std": 0.9,
    "outlier_ratio": 0.03
}

loss_params = {
    "n_epochs": 1000,
    "weight_decay": 1e-3,
    "artl_weight": 0,
    "lambd": 1e-3,
    "k": (1, 2, 3,), # order
    "q": 2,  # exponent
    "M": 100
}

plot_model(model, loss_params=loss_params, data_params=data_params)

n_epochs: 1000
weight_decay: 0.001
artl_weight: 0
h: 45
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100


In [40]:
model.posterior

<bound method LaplaceBNN.posterior of LaplaceBNN(
  (nn): RegNet(
    (linear0): Linear(in_features=1, out_features=192, bias=True)
    (tanh0): Tanh()
    (linear1): Linear(in_features=192, out_features=192, bias=True)
    (tanh1): Tanh()
    (linear2): Linear(in_features=192, out_features=192, bias=True)
    (tanh2): Tanh()
    (linear3): Linear(in_features=192, out_features=1, bias=True)
  )
)>

# BoTorch

In [ ]:
import torch
from botorch.acquisition.logei import qLogExpectedImprovement
from botorch.models.model import Model
from botorch.optim import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.test_functions import Branin
from botorch.utils.transforms import normalize, unnormalize

# Assuming `LaplaceBNN` is already imported
from _src import LaplaceBNN

def run_botorch_with_laplace_bnn():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Define the benchmark function (Branin function here)
    branin = Branin().to(device)

    # Define bounds for the optimization problem
    bounds = torch.tensor([[0.0, 0.0], [1.0, 1.0]], device=device, dtype=torch.float64)

    # Normalize initial data
    train_x = torch.rand(5, 2, device=device, dtype=torch.float64)
    train_y = branin(train_x).unsqueeze(-1)

    # Define the model
    args = {
        "regnet_dims": [10, 10],
        "regnet_activation": "relu",
        "prior_var": 1.0,
        "noise_var": 1e-2,
        "iterative": False,
    }
    laplace_bnn = LaplaceBNN(args, input_dim=2, output_dim=1, device=device)

    # Fit the model
    laplace_bnn.fit(train_x, train_y)

    for iteration in range(10):
        print(f"Iteration {iteration + 1}")

        # Define the acquisition function
        sampler = SobolQMCNormalSampler(sample_shape=torch.Size([500]))
        qEI = qLogExpectedImprovement(model=laplace_bnn, best_f=train_y.max(), sampler=sampler)

        # Optimize the acquisition function to find the next point to evaluate
        candidates, _ = optimize_acqf(
            acq_function=qEI,
            bounds=bounds,
            q=1,
            num_restarts=10,
            raw_samples=50,
        )

        # Evaluate the objective at the selected candidate
        new_x = candidates.detach()
        new_y = branin(new_x).unsqueeze(-1)

        # Update the training data
        train_x = torch.cat([train_x, new_x])
        train_y = torch.cat([train_y, new_y])

        # Refit the model with the new data
        laplace_bnn.fit(train_x, train_y)

        print(f"Optimized candidate: {new_x}")
        print(f"Objective value: {new_y}")

if __name__ == "__main__":
    run_botorch_with_laplace_bnn()


n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 4
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Iteration 1
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 5
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 2
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 6
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 3
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 7
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 4
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 8
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 5
n_epochs:

# Modular botorch

In [1]:
import torch
from botorch.acquisition.logei import qLogExpectedImprovement
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from botorch.models.model import Model
from botorch.optim import optimize_acqf
from botorch.sampling import SobolQMCNormalSampler
from botorch.test_functions import Branin
from botorch.utils.transforms import normalize, unnormalize

# Assuming `LaplaceBNN` is already imported
from _src import LaplaceBNN


def define_benchmark_function(device):
    return Branin().to(device)

def set_device_and_bounds():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bounds = torch.tensor([[0.0, 0.0], [1.0, 1.0]], device=device, dtype=torch.float64)
    return device, bounds

def normalize_initial_data(device, branin):
    train_x = torch.rand(5, 2, device=device, dtype=torch.float64)
    train_y = branin(train_x).unsqueeze(-1)
    return train_x, train_y

def define_and_fit_model(model_class, model_args, train_x, train_y, device):
    model = model_class(model_args, input_dim=2, output_dim=1, device=device)
    model.fit(train_x, train_y)
    return model

def define_acquisition_function(acq_func_class, model, train_y):
    sampler = SobolQMCNormalSampler(sample_shape=torch.Size([500]))
    acq_func = acq_func_class(model=model, best_f=train_y.max(), sampler=sampler)
    return acq_func

def optimization_loop(model, train_x, train_y, bounds, branin, acq_func_class):
    for iteration in range(10):
        print(f"Iteration {iteration + 1}")

        # Define the acquisition function
        acq_func = define_acquisition_function(acq_func_class, model, train_y)

        # Optimize the acquisition function to find the next point to evaluate
        candidates, _ = optimize_acqf(
            acq_function=acq_func,
            bounds=bounds,
            q=1,
            num_restarts=10,
            raw_samples=50,
        )

        # Evaluate the objective at the selected candidate
        new_x = candidates.detach()
        new_y = branin(new_x).unsqueeze(-1)

        # Update the training data
        train_x = torch.cat([train_x, new_x])
        train_y = torch.cat([train_y, new_y])

        # Refit the model with the new data
        model.fit(train_x, train_y)

        print(f"Optimized candidate: {new_x}")
        print(f"Objective value: {new_y}")

def run_botorch_with_model(model_class, model_args, acq_func_class):
    device, bounds = set_device_and_bounds()
    branin = define_benchmark_function(device)
    train_x, train_y = normalize_initial_data(device, branin)
    model = define_and_fit_model(model_class, model_args, train_x, train_y, device)
    optimization_loop(model, train_x, train_y, bounds, branin, acq_func_class)

In [2]:
model_class = LaplaceBNN  # Replace with any other model class if needed
model_args = {
    "regnet_dims": [10, 10],
    "regnet_activation": "relu",
    "prior_var": 1.0,
    "noise_var": 1e-2,
    "iterative": False,
}
acq_func_class = qLogExpectedImprovement  # Replace with any other acquisition function class if needed
run_botorch_with_model(model_class, model_args, acq_func_class)

n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 4
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Iteration 1


/Users/keisukeonoue/ws/hovr-bo/.venv_hovr/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/Users/keisukeonoue/ws/hovr-bo/.venv_hovr/lib/python3.10/site-packages/botorch/optim/optimize.py:568: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)


n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 5
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[1., 1.]], dtype=torch.float64)
Objective value: tensor([[27.7029]], dtype=torch.float64)
Iteration 2
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 6
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 3
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 7
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 4
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 8
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: tensor([[0., 0.]], dtype=torch.float64)
Objective value: tensor([[55.6021]], dtype=torch.float64)
Iteration 5
n_epochs: 1000
weight_decay: 0
artl_weight: 1.0
h: 9
lambd: 0.001
k: (1, 2, 3)
q: 2
M: 100
Optimized candidate: 

### クラスで実装

- 実験の管理をデータフレームで
- 探索空間の次元に応じて自動的に列を追加
- independent か relative のどちらでサンプルしたかを明示する

In [3]:
from abc import ABC, abstractmethod
import torch
from botorch.sampling import SobolQMCNormalSampler
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from botorch.optim import optimize_acqf
from typing import Optional, Tuple
from _src import LaplaceBNN



class Sampler(ABC):
    @abstractmethod
    def sample(self, bounds: torch.Tensor) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        pass

class IndependentSampler(Sampler):
    def __init__(self, n_samples: int, sample_method: str = "sobol", device=None):
        self.n_samples = n_samples
        self.sample_method = sample_method
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def sample(self, bounds: torch.Tensor) -> Tuple[torch.Tensor, None]:
        if self.sample_method == "sobol":
            sobol = torch.quasirandom.SobolEngine(dimension=bounds.shape[1])
            samples = sobol.draw(self.n_samples).to(device=self.device, dtype=bounds.dtype)
            samples = bounds[0] + (bounds[1] - bounds[0]) * samples
        else:  # random
            samples = torch.rand(self.n_samples, bounds.shape[1], device=self.device, dtype=bounds.dtype)
            samples = bounds[0] + (bounds[1] - bounds[0]) * samples
        return samples, None

class RelativeSampler(Sampler):
    def __init__(self, surrogate_model, n_samples: int = 500, device=None):
        self.surrogate = surrogate_model
        self.n_samples = n_samples
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.qmc_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([n_samples]))
    
    def sample(self, bounds: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Create acquisition function using surrogate model
        acq_function = qLogNoisyExpectedImprovement(
            model=self.surrogate,
            X_baseline=self.surrogate.train_inputs[0],
            sampler=self.qmc_sampler,
        )
        
        # Optimize acquisition function
        candidates, acq_values = optimize_acqf(
            acq_function=acq_function,
            bounds=bounds,
            q=1,
            num_restarts=10,
            raw_samples=self.n_samples,
        )
        
        return candidates, acq_values

class BayesianOptimization:
    def __init__(self, 
                 model_class, 
                 model_args,
                 independent_sampler: Optional[IndependentSampler] = None,
                 device=None):
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.bounds = torch.tensor([[0.0, 0.0], [1.0, 1.0]], device=self.device, dtype=torch.float64)
        
        # Initialize independent sampler
        self.independent_sampler = independent_sampler or IndependentSampler(n_samples=5)
        
        # Get initial samples
        self.train_x, _ = self.independent_sampler.sample(self.bounds)
        self.train_y = self._evaluate_objective(self.train_x)
        
        # Initialize surrogate model
        self.model = model_class(model_args, input_dim=2, output_dim=1, device=self.device)
        self.model.fit(self.train_x, self.train_y)
        
        # Initialize relative sampler with surrogate
        self.relative_sampler = RelativeSampler(surrogate_model=self.model)

    def _evaluate_objective(self, x):
        return self.branin(x).unsqueeze(-1)

    def optimize(self, iterations=10):
        for iteration in range(iterations):
            # Sample using relative sampler
            new_x, _ = self.relative_sampler.sample(self.bounds)
            new_y = self._evaluate_objective(new_x)
            
            # Update data and refit model
            self.train_x = torch.cat([self.train_x, new_x])
            self.train_y = torch.cat([self.train_y, new_y])
            self.model.fit(self.train_x, self.train_y)
            
            print(f"Iteration {iteration + 1}")
            print(f"New point: {new_x}")
            print(f"Objective value: {new_y}")



model_args = {
    "regnet_dims": [10, 10],
    "regnet_activation": "relu",
    "prior_var": 1.0,
    "noise_var": 1e-2,
    "iterative": False,
}

bo = BayesianOptimization(
    model_class=LaplaceBNN,
    model_args=model_args,
)
bo.optimize(iterations=10)

AttributeError: 'BayesianOptimization' object has no attribute 'branin'